In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model


In [2]:
dir = 'C:/_dados/_avito/'

In [3]:
train = pd.read_csv(dir + 'train.csv').fillna(0)
test = pd.read_csv(dir + 'test.csv').fillna(0)

print(train.head())
print(train.axes)

        item_id       user_id                 region              city  \
0  b912c3c6a6ad  e00f8ff2eaf9   Свердловская область      Екатеринбург   
1  2dac0150717d  39aeb48f0017      Самарская область            Самара   
2  ba83aefab5dc  91e2f88dd6e3     Ростовская область    Ростов-на-Дону   
3  02996f1dd2ea  bf5cccea572d              Татарстан  Набережные Челны   
4  7c90be56d2ab  ef50846afc0b  Волгоградская область         Волгоград   

  parent_category_name               category_name  \
0          Личные вещи  Товары для детей и игрушки   
1      Для дома и дачи           Мебель и интерьер   
2  Бытовая электроника               Аудио и видео   
3          Личные вещи  Товары для детей и игрушки   
4            Транспорт                  Автомобили   

                       param_1     param_2 param_3                  title  \
0    Постельные принадлежности           0       0  Кокоби(кокон для сна)   
1                       Другое           0       0      Стойка для Одежды   

In [6]:
x_train = train[['price', 'item_seq_number', 'image_top_1']]
x_test = test[['price', 'item_seq_number', 'image_top_1']]
y_train = train.deal_probability
#y_test = test.deal_probability

print(x_train)

             price  item_seq_number  image_top_1
0            400.0                2       1008.0
1           3000.0               19        692.0
2           4000.0                9       3032.0
3           2200.0              286        796.0
4          40000.0                3       2264.0
5           1300.0                9        796.0
6          11000.0              125       2823.0
7            500.0               61        567.0
8            500.0               85        415.0
9            400.0              136         46.0
10         18000.0                6       1396.0
11             1.0                4        119.0
12            70.0               15       1040.0
13          2500.0               47       2849.0
14          5000.0                6       2133.0
15           800.0               86        408.0
16        140000.0              201       1118.0
17           500.0                9        983.0
18          1000.0                4         95.0
19        100000.0  

In [9]:
lrFull = linear_model.LinearRegression()
lrFull.fit(x_train, y_train)
predictions_full = lrFull.predict(x_test)

# mean squared error on full dataset logistic regression
#meanSquaredError = np.sum(np.square(x_test - predictions_full))/predictions_full.size
#rootMeanSquaredError = math.sqrt(meanSquaredError)
#print('Full dataset root mean squared error: ', rootMeanSquaredError)

groupableColumnLabels = ['region', 'city', 'parent_category_name', 'category_name', 'user_type']



In [10]:
groupModelsDict = {}

for groupableColumnLabel in groupableColumnLabels:
    print('Building regression models ' + groupableColumnLabel)
    # split our dataframe into a set of dataframes for each parent_category
    train_grouping = train.groupby(groupableColumnLabel)
    train_groups_dict = {}
    [train_groups_dict.__setitem__(x,train_grouping.get_group(x)) for x in train_grouping.groups]

    # build our set of regression models one for each parent_category_name
    regression_models_dict = {}
    for key, train_group in train_groups_dict.items():
        lr = linear_model.LinearRegression()
        lr.fit(train_group[['price', 'item_seq_number', 'image_top_1']], train_group.deal_probability)
        regression_models_dict[key] = lr

    groupModelsDict[groupableColumnLabel] = regression_models_dict




Building regression models region
Building regression models city
Building regression models parent_category_name
Building regression models category_name
Building regression models user_type


In [13]:
# iterate over all rows in our test data and build a new row of predictions, one for each category column
print('Making predictions')
predictions = []
for index, row in test.iterrows():
    row_data = np.reshape([row['price'], row['item_seq_number'], row['image_top_1']], (1,-1))
    
    rowPredictions = []
    for groupableColumnLabel in groupableColumnLabels:
        groupableColumnValue = row[groupableColumnLabel]
        # if a model is missing for whatever reason pick up the full model to at least get a number
        if groupableColumnValue in regression_models_dict:
            prediction = groupModelsDict[groupableColumnLabel][groupableColumnValue].predict(row_data)
        else:
            prediction = lrFull.predict(row_data)
        rowPredictions.append(min(max(0,prediction[0]),1))

    predictions.append(rowPredictions)



Making predictions


In [26]:
#meanPredictions = [sum(p)/5 for p in predictions] 


#meanPredictions = [np.median(p) for p in predictions]
from scipy import stats
meanPredictions = [stats.gmean(p) for p in predictions]


# mean squared error on individual parent_category logistic regression
#meanSquaredError = np.sum(np.square(x_test - meanPredictions))/len(meanPredictions)
#rootMeanSquaredError = math.sqrt(meanSquaredError)
print(meanPredictions[:100])

C:\Anaconda3\lib\site-packages\scipy\stats\stats.py:305: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


[0.18144076623748662, 0.0933816898604227, 0.2233659307700773, 0.09249090484752788, 0.13733644023733252, 0.15492572725946768, 0.0791600754844388, 0.17046339676101405, 0.175650088066411, 0.09709974111229412, 0.09304700328514405, 0.21051460277528197, 0.15492050849331024, 0.12225830539424173, 0.15267680893663244, 0.18988164967618035, 0.19377470463346094, 0.09470365029541246, 0.09712735352925676, 0.13316863303442925, 0.11321370562228215, 0.17660900385440603, 0.14462338188143686, 0.09670204471459604, 0.1397877799425084, 0.11542825310799325, 0.22118741999822997, 0.21294934610545693, 0.12068708069592995, 0.15014909002261997, 0.09331474734237409, 0.0930531515894683, 0.1334518127947145, 0.14727134267622363, 0.14122268908321622, 0.04168115877748797, 0.19595572331842423, 0.09519251974749804, 0.14322836650816045, 0.12257372290701901, 0.14091995624267603, 0.17524116679945095, 0.14357998662772148, 0.11022044102038325, 0.12872659645172407, 0.16424368410288193, 0.1577988502495538, 0.1121411372636054, 0

In [27]:
with open(dir + 'submissions/cola2.csv', 'w') as f:
    f.write('item_id,deal_probability\n')
    label      = test['item_id'].values
    for label_, predict_ in zip(label, meanPredictions):
        p = str(min(1,max(0,float(predict_))))
        f.write(str(label_) + ',' + p + '\n')